# Book 3 - Random Forest 5k Events Train

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import pickle
import random
import datetime

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.cm as cm
import matplotlib.patheffects as path_effects
from matplotlib import cm as cm2
matplotlib.style.use('ggplot') 
%matplotlib inline

from sklearn import linear_model, datasets
from sklearn.grid_search import GridSearchCV
from sklearn import datasets
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, \
    GradientBoostingClassifier, ExtraTreesClassifier, BaggingRegressor, \
    ExtraTreesRegressor, AdaBoostRegressor, GradientBoostingRegressor, RandomForestRegressor
from sklearn.metrics import confusion_matrix
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from sklearn.metrics import accuracy_score \
#     confusion_matrix, roc_curve, roc_auc_score, precision_score, recall_score
from sklearn.cross_validation import train_test_split, KFold, cross_val_score


matplotlib.style.use('ggplot') 
%matplotlib inline

/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/grid_search.py:43: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)


# Bring in X & T train & test

In [ ]:

# Use this to do train/test split after fit
event_US_id = list(events['display_id'].unique())

import random

# Random list from event_id list of df1 & df2
random.seed(a=47)
rand_list = random.sample(event_US_id,11000)

train_list = rand_list[:10000]
test_list = rand_list[10000:]




In [3]:
with open('X.pkl', 'r') as picklefile:
    X = pickle.load(picklefile)
    


numpy.ndarray

In [7]:
X = X_train.append(X_test)
print('Combined length: {0:,}'.format(len(X)))

Combined length: 25,575


In [8]:
X[23178:23180]

,pltfrm,state,cmpgn_id,adv_id,period,doc_topic,ad_topic
23178,1,NJ,32840,16,overnight,296.0,nan
0,2,TX,6560,1890,overnight,17.0,219.0


In [9]:
X.columns

Index([u'pltfrm', u'state', u'cmpgn_id', u'adv_id', u'period', u'doc_topic',
       u'ad_topic'],
      dtype='object')

In [10]:
start = datetime.datetime.now()

X = pd.get_dummies(X, prefix=['pltfrm', 'state', 'cmpgn_id', 'adv_id',\
                                          'period','doc_topic', 'ad_topic'])

X.drop(['pltfrm_3','state_AK','cmpgn_id_32840','adv_id_1890','period_morning',
       'doc_topic_17.0','ad_topic_91.0'])
finish = datetime.datetime.now()
print(finish - start)

0:00:07.462028


In [11]:
X.head(2)

,pltfrm_1,pltfrm_2,pltfrm_3,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,state_CT,...,ad_topic_91.0,ad_topic_92.0,ad_topic_93.0,ad_topic_94.0,ad_topic_95.0,ad_topic_96.0,ad_topic_97.0,ad_topic_98.0,ad_topic_99.0,ad_topic_nan
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
X_train = X[:23179]

In [13]:
X_train.tail()

,pltfrm_1,pltfrm_2,pltfrm_3,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,state_CT,...,ad_topic_91.0,ad_topic_92.0,ad_topic_93.0,ad_topic_94.0,ad_topic_95.0,ad_topic_96.0,ad_topic_97.0,ad_topic_98.0,ad_topic_99.0,ad_topic_nan
23174,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23175,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23176,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23177,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
23178,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0


In [14]:
X_test = X[23179:]

In [15]:
X_test.head()

,pltfrm_1,pltfrm_2,pltfrm_3,state_AK,state_AL,state_AR,state_AZ,state_CA,state_CO,state_CT,...,ad_topic_91.0,ad_topic_92.0,ad_topic_93.0,ad_topic_94.0,ad_topic_95.0,ad_topic_96.0,ad_topic_97.0,ad_topic_98.0,ad_topic_99.0,ad_topic_nan
0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [16]:
print('X_train length: {0:,}'.format(len(X_train)))
print('X_test  length:  {0:,}'.format(len(X_test)))

X_train length: 23,179
X_test  length:  2,396


In [ ]:
### Y train and Y test    set these up

# Random Forrest Classifier

RFC default settings
* correct predictions: 142
* Precent correct: 31%

RFC on accuracy max (n_estimators = 10, max depth = 5)
* correct predictions: 115
* Precent correct: 25%

RFC on precision max (n_estimators = 20, max depth = 30)
* correct predictions: 154
* Precent correct: 33.7%

In [17]:
rfc = RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

rfc.fit(X_train, Y_train)

#start = datetime.datetime.now()
#finish = datetime.datetime.now()
#print(finish - start)

RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=20, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)

In [18]:
# get probabilities of clicked/not clicked
rf_proba = rfc.predict_proba(X_test)
rf_Y_proba = pd.DataFrame(rf_proba,columns = ['p_no_click','p_click'])

In [19]:
ppp = pd.concat([Disp_test, rf_Y_proba], axis=1)
# post processing of predictor

In [20]:
ppp.tail(7)

,display_id,p_no_click,p_click
2389,16788977,0.878545,0.121455
2390,16838190,0.814677,0.185323
2391,16838190,0.814677,0.185323
2392,16838190,0.802831,0.197169
2393,16838190,0.814677,0.185323
2394,16838190,0.814677,0.185323
2395,16838190,0.773168,0.226832


## Reduce to 1-click per display id

In [21]:
checker = ppp
checker.head(3)

,display_id,p_no_click,p_click
0,65537,0.797993,0.202007
1,65537,0.797993,0.202007
2,65537,0.797993,0.202007


In [22]:
checker['Y_test'] = pd.Series(Y_test)
checker['prediction'] = [0 for _ in range(len(checker))] # puts zeros in place
checker.head()

,display_id,p_no_click,p_click,Y_test,prediction
0,65537,0.797993,0.202007,0,0
1,65537,0.797993,0.202007,0,0
2,65537,0.797993,0.202007,0,0
3,65537,0.797993,0.202007,1,0
4,66286,0.797993,0.202007,1,0


In [23]:
pages = checker['display_id'].unique()
len(pages)

457

In [24]:
# system to use probabilities to choose which page is clicked
# compensates for zero probability across the board, or muliple matching high probabilities

for d_num in pages:  
    indx = 0 # index that becomes click
    disp_list = checker[checker['display_id'] == d_num].index.tolist()

    temp = pd.DataFrame(ppp[ppp['display_id'] == d_num])

    p_max = temp['p_click'].max()

    if p_max == 0:
        indx = random.choice(disp_list) # randomly picks from all matched values
    else:
        counter = 0
        short_list =[]
        for index, row in temp.iterrows():
            if row['p_click'] == p_max:
                counter += 1
                short_list.append(index)
        indx = random.choice(short_list) # randomly picks from all matched values
    checker.set_value(indx, 'prediction', 1)


In [25]:
sum(checker['Y_test'])

457

In [26]:
sum(checker['prediction'])

457

In [27]:
count = 0
for index, row in checker.iterrows():
    if row['prediction'] == 1 and row['Y_test'] == 1:
        count +=1
print('Clicks correctly predicted: {0:,}'.format(count))
print('Percent correctly predicted: {0:%}'.format(float(count)/sum(checker['Y_test'])))

Clicks correctly predicted: 154
Percent correctly predicted: 33.698031%


In [28]:
rf_PARAMETERS = {"max_depth":[5,10,15,30], "n_estimators": [10,20,50,100]}
rf_SCORING = "precision"
rf_model = RandomForestClassifier()
grid = GridSearchCV(rf_model, param_grid=rf_PARAMETERS, scoring=rf_SCORING)
grid.fit(X_train, Y_train)

#look at max features

print grid.best_estimator_
print grid.best_score_

/Users/alex/anaconda/lib/python2.7/site-packages/sklearn/metrics/classification.py:1113: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 due to no predicted samples.
  'precision', 'predicted', average, warn_for)


RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=30, max_features='auto', max_leaf_nodes=None,
            min_impurity_split=1e-07, min_samples_leaf=1,
            min_samples_split=2, min_weight_fraction_leaf=0.0,
            n_estimators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False)
0.374071837056
